<a href="https://colab.research.google.com/github/maheshboj/agenticai_basics/blob/Langchain_components/Agent_With_HITL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.9 MB/s eta 0:00:00


In [7]:
%pip install -qU langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
# ========================================
# LangChain Agent with HITL (Colab Version)
# ========================================
# !pip install langchain langchain-openai langgraph streamlit

import json
from typing import Dict, Any
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
import os
from google.colab import userdata


In [9]:
import json
from typing import Dict, Any
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage

# Added for the new Snowflake tool
import yaml
from langchain_community.utilities import SQLDatabase # Assuming this is where SQLDatabase lives

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY2')  # Replace with your key

# ----- Simple tools -----
@tool
def add(x: int, y: int) -> int:
    """Safely add two integers."""
    return x + y

@tool
def send_email_tool(to: str, subject: str, body: str) -> str:
    """Send an email. This is SENSITIVE and must be approved by a human."""
    return f"✅ Email sent to {to}: '{subject}' - '{body}'"

@tool
def create_snowflake_tables(ddl: str) -> None:
  """Creates tables in Snowflake using the provided DDL. Requires a snowflake_config.yaml file.This is SENSITIVE and must be approved by a human."""
  with open("snowflake_config.yaml", "r") as f:
      config = yaml.safe_load(f)

  sf = config["snowflake"]

  # Build Snowflake connection URI
  connection_uri = (
      f"snowflake://{sf['user']}:{sf['password']}@{sf['account']}/"
      f"{sf['database']}/{sf['schema']}?warehouse={sf['warehouse']}"
  )

  # If role is included
  if "role" in sf:
      connection_uri += f"&role={sf['role']}"

  #print("Snowflake URI:", connection_uri)

  db = SQLDatabase.from_uri(connection_uri)
  db.run(ddl)
  print("Done!" , f"create sf table for {ddl}")

tools = [add, send_email_tool, create_snowflake_tables]

# ----- Model -----
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ----- HITL middleware (simplified for Colab) -----
class SimpleHITL:
    """Simple HITL handler for Colab - pauses and asks via input()"""

    def __init__(self):
        self.pending_tools = []

    def should_interrupt(self, tool_name: str) -> bool:
        # Interrupt for both send_email_tool and create_snowflake_tables
        return tool_name in ["send_email_tool", "create_snowflake_tables"]

    def get_human_decision(self, tool_args: Dict[str, Any]) -> Dict[str, Any]:
        print("\n" + "="*60)
        print("🛑 HUMAN APPROVAL REQUIRED")
        print(f"Tool: {tool_args.get('name', 'unknown')}")
        print("Arguments:")
        for k, v in tool_args.get('args', {}).items():
            print(f"  {k}: {v}")

        while True:
            print("\nOptions:")
            print("1. APPROVE (a)")
            print("2. EDIT args (e)")
            print("3. REJECT (r)")

            choice = input("Enter choice (a/e/r): ").strip().lower()

            if choice == 'a':
                return {"decision": "approve"}
            elif choice == 'r':
                return {"decision": "reject"}
            elif choice == 'e':
                new_args = {}
                for key in tool_args.get('args', {}):
                    new_val = input(f"New {key} (current: {tool_args['args'][key]}): ").strip()
                    if new_val:
                        new_args[key] = new_val
                return {"decision": "edit", "args": new_args}

hitl_handler = SimpleHITL()

# ----- Agent -----
memory = InMemorySaver()
agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,
)

# ----- Run function with HITL -----
def run_agent_with_hitl(user_input: str, thread_id: str = "colab-thread"):
    config = {"configurable": {"thread_id": thread_id}}

    print(f"\n🤖 Agent thinking about: '{user_input}'")
    # Fix: The agent expects a list of messages in the 'messages' key in the input state.
    # Convert the user_input string into a HumanMessage object.
    initial_messages = [HumanMessage(content=user_input)]
    stream = agent.stream({"messages": initial_messages}, config, stream_mode="values")

    final_output = None
    for i, chunk in enumerate(stream):
        print(f"\n--- Step {i+1} ---")
        print(json.dumps(chunk, indent=2, default=str))

        # Check for tool calls that need HITL
        if "messages" in chunk and chunk["messages"]:
            last_msg = chunk["messages"][-1]
            # LangGraph agents often return 'tool_calls' as a list of dicts directly
            # or wrapped in a 'BaseMessage' object. We need to adapt this check.
            tool_calls_data = []
            if isinstance(last_msg, dict) and 'tool_calls' in last_msg:
                 # This path might be for dict output directly from agent state
                tool_calls_data = last_msg['tool_calls']
            elif hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                # This path is for BaseMessage objects
                tool_calls_data = last_msg.tool_calls

            for tool_call in tool_calls_data:
                tool_name = tool_call['name']
                if hitl_handler.should_interrupt(tool_name):
                        print("\n🔍 HITL TRIGGERED!")
                        decision = hitl_handler.get_human_decision({
                            "name": tool_name,
                            "args": dict(tool_call['args'])
                        })

                        # Simulate resume logic (in real app, you'd update state)
                        print(f"\nHuman decision: {decision}")
                        if decision["decision"] == "approve":
                            print("✅ Proceeding with original args")
                        elif decision["decision"] == "reject":
                            print("❌ Skipping tool call")
                            continue
                        elif decision["decision"] == "edit":
                            print(f"✏️ Using edited args: {decision.get('args', {})}")

    print("\n🎉 Agent complete!")
    return final_output

# ----- RUN THE DEMO -----
if __name__ == "__main__":
    print("🚀 LangChain HITL Agent Demo (Colab)")
    print("="*50)

    # Test 1: Safe tool (add)
    print("\n📝 Test 1: Safe math operation")
    run_agent_with_hitl("What is 15 + 27?")

    # Test 2: Sensitive email operation
    print("\n📧 Test 2: Sensitive email operation")
    run_agent_with_hitl("Send an email to alice@example.com about tomorrow's meeting")

    # Test 3: Sensitive Snowflake table creation
    print("\n❄️ Test 3: Sensitive Snowflake table creation")
    snowflake_ddl = """CREATE TABLE IF NOT EXISTS my_test_table (
        id INT,
        name VARCHAR(255),
        created_at TIMESTAMP
    );"""
    run_agent_with_hitl(f"Create a Snowflake table with the following DDL: {snowflake_ddl}")


In [12]:
    snowflake_ddl = """CREATE TABLE IF NOT EXISTS my_test_table (
        id INT,
        name VARCHAR(255),
        created_at TIMESTAMP
    );"""

In [13]:
run_agent_with_hitl(f"Create a Snowflake table with the following DDL: {snowflake_ddl}")


🤖 Agent thinking about: 'Create a Snowflake table with the following DDL: CREATE TABLE IF NOT EXISTS my_test_table (
    id INT,
    name VARCHAR(255),
    created_at TIMESTAMP
);'

--- Step 1 ---
{
  "messages": [
    "content='What is 15 + 27?' additional_kwargs={} response_metadata={} id='9eb98940-28f3-4d33-9775-95a304da18c0'",
    "content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 140, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_50906f2aac', 'id': 'chatcmpl-ChbuTTimCS7fws65kmNxKY8gNmhbA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--ecd297db-f10f-487d-a7da-186595930faa-0' tool_calls=[{'name': '

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:snowflake

In [ ]:
# ----- RUN THE DEMO -----
if __name__ == "__main__":
    print("🚀 LangChain HITL Agent Demo (Colab)")
    print("="*50)

    # Test 1: Safe tool (add)
    print("\n📝 Test 1: Safe math operation")
    run_agent_with_hitl("What is 15 + 27?")

    # Test 2: Sensitive email operation
    print("\n📧 Test 2: Sensitive email operation")
    run_agent_with_hitl("Send an email to alice@example.com about tomorrow's meeting")